# BAYESIAN MODEL

In [31]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

from numpy.random import randn
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sb

In [32]:
%matplotlib inline
rcParams['figure.figsize'] = 20, 9
sb.set_style('whitegrid')

In [33]:
import pymc3 as pm
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
from PIL import Image

In [34]:
df =pd.read_csv ('C:/Users/darog/NOTEBOOKS/PPUC/To GitHub/df.csv', error_bad_lines=False)
df.head()

,car_id,car_make,car_model,car_year,car_mileage,car_price,car_transmission,car_driveTrain,car_vendor,car_location,car_engine
0,73531922,Toyota,Corolla-LE,2015,23036,13298,Automatic,FWD,Chantilly,VA 20151,1.8 4 Cylinder Sequential-Port F.I.
1,72354903,Jeep,Wrangler-Unlimted-Sahara,2012,73619,26995,AUTOMATIC,4WD,Baltimore,MD 21215,3.6L 6 CYL. GAS
2,73704666,Honda,Civic-LX,2014,27784,14097,Automatic,FWD,Ellicott City,MD 21043,1.80L I4
3,73566743,Jeep,Grand-Cherokee-Limited,2014,36636,27899,8-Speed Automatic,4WD,Palmyra,PA 17078,3.6L V6 Flex Fuel 24V VVT
4,73745994,Hyundai,Santa-Fe-Sport-24L,2013,48568,18981,A,AWD,Doylestown,PA 18902,2.40L I4


In [35]:
#df.groupby('car_make').count()

In [36]:
car_make_names = df.car_make.unique()
car_make_names 

array(['Toyota', 'Jeep', 'Honda', 'Hyundai', 'GMC', 'Chevrolet', 'Lincoln',
       'Mazda', 'Kia', 'Buick', 'Volkswagen', 'Infiniti', 'Subaru',
       'Cadillac', 'Ford', 'BMW', 'Acura', 'Lexus', 'Dodge', 'Chrysler',
       'Maserati', 'Jaguar', 'FIAT', 'Nissan', 'Audi', 'Volvo',
       'Land-Rover', 'Mitsubishi', 'Ram', 'MINI', 'Mercedes-Benz', 'Scion',
       'TESLA', 'Suzuki', 'Porsche', 'Lamborghini', 'Bentley', 'Mercury',
       'Saturn', 'Aston-Martin', 'Hummer', 'Genesis', 'Ferrari', 'McLaren',
       'Rolls-Royce', 'Smart', 'Pontiac', 'Saab', 'Isuzu', 'Oldsmobile',
       'Maybach', 'INFINITI', 'Land Rover', 'Aston Martin', 'RAM',
       'Triumph', 'MG', 'Tesla', 'Alfa Romeo', 'smart'], dtype=object)

In [37]:
# Make a column 'ymm' containing year make model.
def ymm_cc(df):

    # making all of these string, as some are numeric
    year = str(df.car_year)
    make = str(df.car_make)
    model = str(df.car_model)
    ymm = ' '.join([year,make,model])
    return ymm
    
df['ymm'] = df.apply(ymm_cc,axis=1)
df['ymm'].head()

0                2015 Toyota Corolla-LE
1    2012 Jeep Wrangler-Unlimted-Sahara
2                   2014 Honda Civic-LX
3      2014 Jeep Grand-Cherokee-Limited
4       2013 Hyundai Santa-Fe-Sport-24L
Name: ymm, dtype: object

In [38]:
#df.head()

,car_id,car_make,car_model,car_year,car_mileage,car_price,car_transmission,car_driveTrain,car_vendor,car_location,car_engine,ymm
0,73531922,Toyota,Corolla-LE,2015,23036,13298,Automatic,FWD,Chantilly,VA 20151,1.8 4 Cylinder Sequential-Port F.I.,2015 Toyota Corolla-LE
1,72354903,Jeep,Wrangler-Unlimted-Sahara,2012,73619,26995,AUTOMATIC,4WD,Baltimore,MD 21215,3.6L 6 CYL. GAS,2012 Jeep Wrangler-Unlimted-Sahara
2,73704666,Honda,Civic-LX,2014,27784,14097,Automatic,FWD,Ellicott City,MD 21043,1.80L I4,2014 Honda Civic-LX
3,73566743,Jeep,Grand-Cherokee-Limited,2014,36636,27899,8-Speed Automatic,4WD,Palmyra,PA 17078,3.6L V6 Flex Fuel 24V VVT,2014 Jeep Grand-Cherokee-Limited
4,73745994,Hyundai,Santa-Fe-Sport-24L,2013,48568,18981,A,AWD,Doylestown,PA 18902,2.40L I4,2013 Hyundai Santa-Fe-Sport-24L


In [40]:
# number of iterations should be greater than 100 but due to computation speed and memory in my machine, it's been 
# set up at 100. Once the code works set it up at 100000

niter = 100

car_make_names = df.car_make.unique()

# we have chosen Chevrolet, however for different cars analysis the car_make_names must be set up to the car make we 
# are interested on

car_make_names = ['Chevrolet']

####################################################################################################################
# THIS BLOCK IS SUPPOSED TO LET US SELECT THE CAR MODEL AND YEAR WE WANT TO ANALYZE !!!!

for make in car_make_names:
    # let's create some empty dictionaries to store the results
    bayes_by_yearmakemodel = {}
    bayes_by_car_model = {}
    # final dataframe
    bayes_make_model_year = pd.DataFrame()

    # this is the datframe only for the car make we are analyzing, for this particular example it'll be Chevrolet, 
    # all the Chevrolet cars are stored in the dataframe df_make
    df_make = df[df.car_make == make]
    
    # we are giving a numerical value for each car_model, we define the model info for each make. For example Malibu-LT
    # has been assigned with the code 95
    model_coder = LabelEncoder()
    df_make['model_code'] = model_coder.fit_transform(df_make.car_model)
    model_codes = df_make.model_code.unique()
    model_codes_idx = df_make.model_code.values
    num_model_codes = len(model_codes)
    model_names = df_make.car_model.unique()
     
    #for model in model_names:
    for mdl,mdl_i in enumerate(model_codes[0:1]):
        model = str(model_coder.inverse_transform(mdl))
        bayes_by_year = {}
        # we will select the model code under consideration by changing the index in model_codes[0:1]
    
        # once the car model under consideration is selected, the results will be stored in the dataframe df_model
        df_model= df_make[df_make.car_model == model]
        
        # the same thing we did for the model code, a numerical year code will be given for each year and for each make
        year_le = LabelEncoder()
        
        df_model['year_code'] = year_le.fit_transform(df_model.car_year)
        
        years = df_model.year_code.unique()
        
        year_codes_idx = df_model.car_year.values
        
        num_years = len(df_model.year_code.unique()) 
                
        year_codes = df_model.year_code.unique()
        
        # run for each year of each model, this loop for is gonna help us to keep only the year we want to base our 
        # analyssis on
        for yr,yr_1 in enumerate(year_codes[0:3]):

            year = str(year_le.inverse_transform(yr))

            # subset the data again
            df_year = df_model[df_model.car_year == int(year)]

            # Let's take a look on the car we are going to study
            print "The car under consideration for our analysis is %s %s %s " % (year,make,model)

##################################################################################################################

            with pm.Model() as price_ymm:

                # define the priors and distributions for:

                # intercept priors and distribution: model 

                # set mu for the intercept to the mean price
                intercept_mu = df_year.car_price.mean()

                intercept_model_mean = pm.Normal('intercept_model_mean', mu=intercept_mu, sd=10000.)
                intercept_model_sd = pm.Normal('intercept_model_sd',mu=intercept_mu, sd=10000.)
                intercept_model_overall = pm.Normal('intercept_model_overall',
                                            mu=intercept_model_mean, sd=intercept_model_sd, shape=num_model_codes)

                # miles priors: model
                miles_model_mean = pm.Normal('miles_model_mean', mu=0, sd=10000.)
                miles_model_sd = pm.Gamma('miles_model_sd', mu=0.0001, sd=10000.)
                # miles distribuitions
                miles_model_overall = pm.Normal('miles_model_overall',mu=miles_model_mean,
                                          sd=miles_model_sd, shape=num_model_codes)

                # regression formula
                price_est = (intercept_model_overall[model_codes_idx] +
                             miles_model_overall[model_codes_idx]*df_year.car_mileage.values)

                # model error
                model_error = pm.Uniform('model_error', lower=0, upper=100000)

                # calculate likelihood
                price_like = pm.Normal('price_like', mu=price_est, sd=model_error, observed=df_year.car_price.values)

                step = pm.Metropolis()

                # run
                trace = pm.sample(niter,step=step)

                #print '\n'          

#####################################################################################################################
            
                #don't keep all the traces
                bayes_by_year[year] = trace[niter/2:]

            # outside the Bayes model now
            # create and populate the dataframe
            # define lists that will be used when creating the new dataframe
            yr_names = []
            yr_codes = []
            mk_names = []
            mdl_names = []
            mdl_codes = []
            ymms = []
            prob_dists = []

            # go through each trim level
            for mdl, mdl_i in enumerate(model_codes):
            
                # now get the year, model values back from using the inverse encoder
                yr_e = year_le.inverse_transform(yr)
                mdl_e = str(model_coder.inverse_transform(mdl))
                
                # create ymm value
                ymm = ' '.join([str(yr_e),make,mdl_e])

                # if the ymmt isn't in the parent set, then it doesn't
                # need to be in the final df
                if len(df[df.ymm == ymm]) > 0:

                    # fill lists that will be used when creating the new dataframe
                    ymms.append(ymm)
                    yr_names.append(yr_e)
                    yr_codes.append(yr)
                    mk_names.append(make)
                    mdl_names.append(mdl_e)
                    mdl_codes.append(mdl)
                    
                    # get the price probability dist for each ymmt from the bayes files
                    prob =      (bayes_by_year[year]['intercept_model_overall'][:,mdl] +
                                 bayes_by_year[year]['miles_model_overall'][:,mdl])

                    prob_dists.append(prob)

                else:
                    pass

            # put it all in a datafram
            year_df = pd.DataFrame({
                    'ymm':ymms,
                    'makeName':mk_names,
                    'modelYear':yr_names,
                    'year_code':yr_codes,
                    'modelName':mdl_names,
                    'model_code':mdl_codes,
                    'prob_dists':prob_dists
                })

            # update the dataframe
            bayes_make_model_year = pd.concat((bayes_make_model_year,year_df))

            # save traces with model
            bayes_by_model[model] = bayes_by_year

        # save traces by make
        bayes_by_ymm[make] = bayes_by_model
        #print bayes_by_ymm[make].keys()

    # reset the df index before saving
    bayes_make_model_year.reset_index(inplace=True,drop=True)
    # directory for saving the files
    file_dir = 'C:/Users/darog/Desktop/bayes/'
    
    # save the dataframe
    #df_file = file_dir+str(make)+'_df.pgz'
    #save_zipped_pickle(bayes_make_model_year,df_file)

    # file name for the traces
    #trace_file = file_dir +str(make)+'_bayes.pgz'
    # save the traces
    # pickle the traces to save the output
    #save_zipped_pickle(bayes_by_ymm,trace_file)

  

C:\Users\darog\Anaconda2\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\darog\Anaconda2\lib\site-packages\ipykernel\__main__.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


The car under consideration for our analysis is 2007 Chevrolet Avalanche-1500-LTZ 


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [02:24<00:00,  4.17it/s]


NameError: name 'bayes_by_model' is not defined

In [30]:
#df_model

In [28]:
df_year

,car_id,car_make,car_model,car_year,car_mileage,car_price,car_transmission,car_driveTrain,car_vendor,car_location,car_engine,model_code,year_code
19052,701954170,Chevrolet,Malibu,2015,8827,17000,Automatic,FWD,Washington,Washington,NaN,89,6
19112,705817513,Chevrolet,Malibu,2015,10739,20500,Automatic,FWD,Washington,Washington,NaN,89,6
19568,705684090,Chevrolet,Malibu,2015,21151,14500,Automatic,FWD,Washington,Washington,NaN,89,6
20246,705684046,Chevrolet,Malibu,2015,34706,13750,Automatic,FWD,Washington,Washington,NaN,89,6
20502,704462843,Chevrolet,Malibu,2015,42172,15000,Automatic,FWD,Washington,Washington,NaN,89,6
20792,703742346,Chevrolet,Malibu,2015,8180,17725,6-Speed Automatic,FWD,Temple Hills,Temple Hills,NaN,89,6
20854,700559453,Chevrolet,Malibu,2015,22988,14000,6-Speed Automatic,FWD,Temple Hills,Temple Hills,NaN,89,6
20962,701805778,Chevrolet,Malibu,2015,38739,13990,6-Speed Automatic,FWD,Temple Hills,Temple Hills,NaN,89,6
21002,701241605,Chevrolet,Malibu,2015,43277,14490,6-Speed Automatic,FWD,Temple Hills,Temple Hills,NaN,89,6
21405,701253329,Chevrolet,Malibu,2015,32541,14913,Automatic,FWD,Marlow Heights,Marlow Heights,NaN,89,6


In [31]:
#year
#car_make_names
#df_make
#df_make['model_code']
#model_codes.sum()
#df_model['year_code']

In [87]:
#df_model['car_year'].unique()
#df_model

In [32]:
#df_make.columns

In [29]:
#df[['car_model','car_make','car_price']].sort_values(by='car_price',ascending=True).head(20)

In [35]:
#df_model